In [29]:
"""
Training Data Map:

[
    {"id": 1,
     "name": "1.jpg",
     "FC_vector": ,
     "POOL_vector": 
    }
]
"""

import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import csv

FC_file = "./../all/data/features_train/features_resnet1000_train.csv"
POOL_file = "./../all/data/features_train/features_resnet1000intermediate_train.csv"

test_FC_file = "./../all/data/features_test/features_resnet1000_test.csv"
test_POOL_file = "./../all/data/features_test/features_resnet1000intermediate_test.csv"


train_data_map = []

with open(FC_file) as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        img_path = row[0]
        img_name = img_path.split("/")[1]
        img_id = int(img_name.split(".")[0])
        img_fc_v = list(map(lambda x: float(x), row[1:]))
        obj = {}
        obj["id"] = img_id
        obj["name"] = img_name
        obj["FC_vector"] = img_fc_v
        train_data_map.append(obj)


test_data_map = []

with open(test_FC_file) as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        img_path = row[0]
        img_name = img_path.split("/")[1]
        img_id = int(img_name.split(".")[0])
        img_fc_v = list(map(lambda x: float(x), row[1:]))
        obj = {}
        obj["id"] = img_id
        obj["name"] = img_name
        obj["FC_vector"] = img_fc_v
        test_data_map.append(obj)

train_data_map = sorted(train_data_map, key=lambda x: int(x["id"]))
test_data_map = sorted(test_data_map, key=lambda x: int(x["id"]))

with open(POOL_file) as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        img_path = row[0]
        img_name = img_path.split("/")[1]
        img_id = int(img_name.split(".")[0])
        img_pool_v = list(map(lambda x: float(x), row[1:]))
        train_data_map[img_id]["POOL_vector"] = img_pool_v

with open(test_POOL_file) as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        img_path = row[0]
        img_name = img_path.split("/")[1]
        img_id = int(img_name.split(".")[0])
        img_pool_v = list(map(lambda x: float(x), row[1:]))
        test_data_map[img_id]["POOL_vector"] = img_pool_v

# print(train_data_map[2])
with open('./data/12_1_train_img_feature_map_py27.pkl', 'wb') as f:
    pickle.dump(train_data_map, f, protocol=2)
    
with open('./data/12_1_test_img_feature_map_py27.pkl', 'wb') as f:
    pickle.dump(test_data_map, f, protocol=2)

print("finish writing...")

finish writing...


In [5]:
"""
Prep training data label, turning all description into BOW. 
Train Data Label(BOW):
[
    {
      "id": 1,
      "name": "1.txt",
      "description_complete": "<all five strings concated>",
      "description_n": "<all five, filter into only nouns>",
      "description_n_v_adj": "..",
      "description_v_adj": "...",
      "BOW_n_vector": ,
      "BOW_n_v_adj_vector": ,
      "BOW_v_adj_vector": 
    }
]

with corresponding BOW dictionary mapping.
"""
from tqdm import tqdm
from nltk.tag import pos_tag
import os

train_img_description = "./../all/data/descriptions_train/"
test_img_description = "./../all/data/descriptions_test/"

current_path = test_img_description ## CHANGE HERE!

In [15]:
"""
Text preprocessing.
"""
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

nltk.download('stopwords')

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    words = stem_words(words)
    words = lemmatize_verbs(words)
    return words

# words = normalize(words)

[nltk_data] Downloading package stopwords to /Users/EL-
[nltk_data]     CAPITAN-2016/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [22]:
"""
Generate and save training dataset BOW vector. 
"""
from gensim import corpora
import pickle
def generate_train_description_bow():
    current_path = train_img_description
    BOW_vector_map = []
    for n in tqdm(os.listdir(current_path)):
        if n.endswith(".txt"):
            txt_file = current_path + n
            with open(txt_file) as f:
                content = f.readlines()
                content = [x.strip() for x in content]
                description_complete = " ".join(content)
            obj = {}
            f_id = int(n.split(".")[0])
            obj["id"] = f_id
            obj["name"] = n
            obj["description_complete"] = description_complete
            tagged = pos_tag(description_complete.split())
            description_n = " ".join([word for word,pos in tagged if pos == 'NN'])
            description_v_adj = " ".join([word for word,pos in tagged if pos == 'VB' or pos == 'JJ'])
            description_n_v_adj = " ".join([word for word,pos in tagged if pos == 'NN' or pos == 'VB' or pos == 'JJ'])
            obj["description_n"] = description_n
            obj["description_v_adj"] = description_v_adj
            obj["description_n_v_adj"] = description_n_v_adj
            
            description_normalized = normalize(description_complete.split())
            obj["description_normalized"] = " ".join(description_normalized)
            BOW_vector_map.append(obj)
    
    all_description_n = list(map(lambda x: x["description_normalized"] , BOW_vector_map))
    texts = [[w for w in d.split()] for d in all_description_n]
    all_tokens = sum(texts, [])
    tokens_once = set(word for word in set(all_tokens) if all_tokens.count(word) == 1)
    texts = [[w for w in t if w not in tokens_once] for t in texts]
    print(texts[0])
    dictionary = corpora.Dictionary(texts)
    dictionary.save("./data/12_2_[train]_description_normalized_only_>1.dict")
    # dictionary = corpora.Dictionary.load("./TRAIN_only_nouns_>30.dict")
    print(dictionary)
    
    for i, t in tqdm(enumerate(texts)):
        # n has no punctuation.
        vec_n = dictionary.doc2bow(t)
        hot_encoding = [0] * (len(dictionary))
        for v in vec_n:
            hot_encoding[v[0]] = 1
        BOW_vector_map[i]['BOW_all_normalized_vector'] = hot_encoding

    BOW_vector_map = sorted(BOW_vector_map, key=lambda x: int(x['id']))
    with open('./data/12_2_[train]_description_normalized_only_>1_vector_py27.pkl', 'wb') as f:
        pickle.dump(BOW_vector_map, f, protocol=2)
    print("Finish Writing...")
    
generate_train_description_bow()



  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 8/10000 [00:00<02:13, 74.67it/s]

  0%|          | 16/10000 [00:00<02:13, 74.70it/s]

  0%|          | 25/10000 [00:00<02:08, 77.87it/s]

  0%|          | 34/10000 [00:00<02:04, 80.23it/s]

  0%|          | 44/10000 [00:00<01:58, 83.78it/s]

  1%|          | 54/10000 [00:00<01:56, 85.55it/s]

  1%|          | 63/10000 [00:00<01:55, 85.97it/s]

  1%|          | 72/10000 [00:00<01:55, 86.04it/s]

  1%|          | 81/10000 [00:00<01:55, 85.70it/s]

  1%|          | 90/10000 [00:01<01:55, 85.47it/s]

  1%|          | 99/10000 [00:01<01:55, 85.77it/s]

  1%|          | 108/10000 [00:01<01:54, 86.22it/s]

  1%|          | 117/10000 [00:01<01:53, 87.28it/s]

  1%|▏         | 126/10000 [00:01<01:52, 87.44it/s]

  1%|▏         | 136/10000 [00:01<01:51, 88.86it/s]

  1%|▏         | 146/10000 [00:01<01:48, 90.55it/s]

  2%|▏         | 156/10000 [00:01<01:49, 90.00it/s]

  2%|▏         | 165/10000 [00:01<01:50, 89.36it/s]

  2%|▏       

 27%|██▋       | 2659/10000 [00:29<01:24, 87.13it/s]

 27%|██▋       | 2669/10000 [00:29<01:23, 87.66it/s]

 27%|██▋       | 2678/10000 [00:29<01:23, 87.91it/s]

 27%|██▋       | 2687/10000 [00:29<01:24, 86.54it/s]

 27%|██▋       | 2696/10000 [00:30<01:26, 84.76it/s]

 27%|██▋       | 2705/10000 [00:30<01:25, 85.67it/s]

 27%|██▋       | 2715/10000 [00:30<01:23, 87.03it/s]

 27%|██▋       | 2725/10000 [00:30<01:21, 89.23it/s]

 27%|██▋       | 2735/10000 [00:30<01:20, 90.54it/s]

 27%|██▋       | 2745/10000 [00:30<01:23, 86.96it/s]

 28%|██▊       | 2754/10000 [00:30<01:23, 87.17it/s]

 28%|██▊       | 2764/10000 [00:30<01:21, 88.61it/s]

 28%|██▊       | 2773/10000 [00:30<01:21, 88.17it/s]

 28%|██▊       | 2782/10000 [00:31<01:24, 85.20it/s]

 28%|██▊       | 2792/10000 [00:31<01:22, 87.26it/s]

 28%|██▊       | 2802/10000 [00:31<01:21, 88.65it/s]

 28%|██▊       | 2812/10000 [00:31<01:19, 90.41it/s]

 28%|██▊       | 2822/10000 [00:31<01:18, 91.02it/s]

 28%|██▊       | 2832/10000 

 53%|█████▎    | 5277/10000 [00:59<00:57, 82.36it/s]

 53%|█████▎    | 5286/10000 [00:59<00:57, 82.56it/s]

 53%|█████▎    | 5295/10000 [00:59<00:57, 81.95it/s]

 53%|█████▎    | 5304/10000 [00:59<00:57, 82.23it/s]

 53%|█████▎    | 5313/10000 [00:59<00:56, 83.36it/s]

 53%|█████▎    | 5322/10000 [00:59<00:55, 83.59it/s]

 53%|█████▎    | 5331/10000 [00:59<00:55, 84.07it/s]

 53%|█████▎    | 5341/10000 [00:59<00:54, 85.99it/s]

 54%|█████▎    | 5351/10000 [00:59<00:52, 87.78it/s]

 54%|█████▎    | 5361/10000 [00:59<00:52, 89.08it/s]

 54%|█████▎    | 5371/10000 [01:00<00:51, 89.86it/s]

 54%|█████▍    | 5380/10000 [01:00<00:51, 89.58it/s]

 54%|█████▍    | 5389/10000 [01:00<00:52, 88.39it/s]

 54%|█████▍    | 5398/10000 [01:00<00:52, 87.01it/s]

 54%|█████▍    | 5408/10000 [01:00<00:52, 87.74it/s]

 54%|█████▍    | 5417/10000 [01:00<00:52, 88.12it/s]

 54%|█████▍    | 5426/10000 [01:00<00:53, 86.17it/s]

 54%|█████▍    | 5435/10000 [01:00<00:52, 86.78it/s]

 54%|█████▍    | 5444/10000 

 79%|███████▉  | 7890/10000 [01:28<00:23, 91.56it/s]

 79%|███████▉  | 7900/10000 [01:28<00:23, 90.25it/s]

 79%|███████▉  | 7910/10000 [01:28<00:23, 89.19it/s]

 79%|███████▉  | 7919/10000 [01:28<00:23, 87.45it/s]

 79%|███████▉  | 7928/10000 [01:28<00:23, 87.51it/s]

 79%|███████▉  | 7937/10000 [01:29<00:23, 86.91it/s]

 79%|███████▉  | 7947/10000 [01:29<00:23, 88.47it/s]

 80%|███████▉  | 7957/10000 [01:29<00:22, 89.10it/s]

 80%|███████▉  | 7966/10000 [01:29<00:22, 88.64it/s]

 80%|███████▉  | 7975/10000 [01:29<00:23, 87.49it/s]

 80%|███████▉  | 7984/10000 [01:29<00:23, 87.27it/s]

 80%|███████▉  | 7993/10000 [01:29<00:23, 86.52it/s]

 80%|████████  | 8002/10000 [01:29<00:23, 83.36it/s]

 80%|████████  | 8012/10000 [01:29<00:23, 85.74it/s]

 80%|████████  | 8021/10000 [01:29<00:23, 85.92it/s]

 80%|████████  | 8031/10000 [01:30<00:22, 87.33it/s]

 80%|████████  | 8041/10000 [01:30<00:22, 88.96it/s]

 80%|████████  | 8050/10000 [01:30<00:21, 89.22it/s]

 81%|████████  | 8059/10000 

['teddy', 'bear', 'clothing', 'hanging', 'clothes', 'lines', 'outside', 'window', 'stuffed', 'toy', 'hanging', 'laundry', 'line', 'items', 'left', 'air', 'dry', 'clothes', 'teddy', 'bear', 'pinned', 'outdoors', 'clothes', 'line', 'stuffed', 'bear', 'hanging', 'window']




0it [00:00, ?it/s]

1423it [00:00, 14229.00it/s]



Dictionary(5816 unique tokens: ['air', 'bear', 'clothes', 'clothing', 'dry']...)


2357it [00:00, 12297.09it/s]

3500it [00:00, 11366.50it/s]

4904it [00:00, 11589.95it/s]

6290it [00:00, 11756.15it/s]

7675it [00:00, 7988.88it/s] 

9052it [00:00, 8879.05it/s]

10000it [00:01, 9616.82it/s]

Finish Writing...


In [26]:
"""
For TEST_BOW_vector_map_5452.pkl
"""
def generate_test_description_bow():
    current_path = test_img_description
    BOW_vector_map = []
    for n in tqdm(os.listdir(current_path)):
        if n.endswith(".txt"):
            txt_file = current_path + n
            with open(txt_file) as f:
                content = f.readlines()
                content = [x.strip() for x in content]
                description_complete = " ".join(content)
            obj = {}
            f_id = int(n.split(".")[0])
            obj["id"] = f_id
            obj["name"] = n
            obj["description_complete"] = description_complete
            tagged = pos_tag(description_complete.split())
            description_n = " ".join([word for word,pos in tagged if pos == 'NN'])
            description_v_adj = " ".join([word for word,pos in tagged if pos == 'VB' or pos == 'JJ'])
            description_n_v_adj = " ".join([word for word,pos in tagged if pos == 'NN' or pos == 'VB' or pos == 'JJ'])
            obj["description_n"] = description_n
            obj["description_v_adj"] = description_v_adj
            obj["description_n_v_adj"] = description_n_v_adj
            
            description_normalized = normalize(description_complete.split())
            obj["description_normalized"] = " ".join(description_normalized)
            BOW_vector_map.append(obj)
    BOW_vector_map = sorted(BOW_vector_map, key=lambda x:x["id"])
    all_description_n = list(map(lambda x: x["description_normalized"] , BOW_vector_map))
    texts = [[w for w in d.split()] for d in all_description_n]
    dictionary = corpora.Dictionary.load("./data/11_30_[train]_description_normalized_only_>1.dict")
    print(dictionary)

    for i, t in tqdm(enumerate(texts)):
        # n has no punctuation.
        vec_n = dictionary.doc2bow(t)
        hot_encoding = [0] * (len(dictionary))
        for v in vec_n:
            hot_encoding[v[0]] = 1
        BOW_vector_map[i]['BOW_all_normalized_vector'] = hot_encoding

    print(BOW_vector_map[0])

    with open('./data/11_30_[test]_description_normalized_only_>1_vector_py27.pkl', 'wb') as f:
        pickle.dump(BOW_vector_map, f, protocol=2)

generate_test_description_bow()



  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 8/2000 [00:00<00:26, 74.86it/s]

  1%|          | 18/2000 [00:00<00:24, 79.88it/s]

  1%|▏         | 27/2000 [00:00<00:24, 82.01it/s]

  2%|▏         | 38/2000 [00:00<00:22, 87.01it/s]

  2%|▏         | 48/2000 [00:00<00:21, 89.32it/s]

  3%|▎         | 58/2000 [00:00<00:21, 92.03it/s]

  3%|▎         | 68/2000 [00:00<00:20, 92.31it/s]

  4%|▍         | 79/2000 [00:00<00:20, 95.33it/s]

  4%|▍         | 89/2000 [00:00<00:20, 95.28it/s]

  5%|▍         | 99/2000 [00:01<00:20, 93.72it/s]

  5%|▌         | 109/2000 [00:01<00:20, 91.80it/s]

  6%|▌         | 119/2000 [00:01<00:20, 90.34it/s]

  6%|▋         | 128/2000 [00:01<00:21, 88.54it/s]

  7%|▋         | 139/2000 [00:01<00:20, 92.35it/s]

  8%|▊         | 150/2000 [00:01<00:19, 95.67it/s]

  8%|▊         | 161/2000 [00:01<00:18, 97.27it/s]

  9%|▊         | 171/2000 [00:01<00:19, 95.17it/s]

  9%|▉         | 181/2000 [00:01<00:18, 96.38it/s]

 10%|▉         | 192/2000 [00:0

Dictionary(5816 unique tokens: ['air', 'bear', 'clothes', 'clothing', 'dry']...)
{'id': 0, 'name': '0.txt', 'description_complete': 'A woman walking down a street past a doorway. a woman is walking past a doorway on a sidewalk A woman talking on a cell phone checks her watch. A woman talks on a cell phone and checks her watch while walking on a sidewalk. A woman wearing blue is on the phone and walking along a sidewalk in front of a building with black planters on both sides of the entrance.', 'description_n': 'woman street doorway. woman doorway sidewalk woman cell phone watch. woman cell phone watch sidewalk. woman blue phone walking sidewalk front building entrance.', 'description_v_adj': 'black', 'description_n_v_adj': 'woman street doorway. woman doorway sidewalk woman cell phone watch. woman cell phone watch sidewalk. woman blue phone walking sidewalk front building black entrance.', 'description_normalized': 'woman walking street past doorway woman walking past doorway sidewalk 

In [27]:
dictionary = corpora.Dictionary.load("./data/11_30_[train]_description_normalized_only_>1.dict")
print(dictionary[19])

blue
